# SAINT

In [ ]:
pip install pandas scikit-learn openml torch numpy

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.0/158.0 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.9/93.9 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 26.3 MB/s eta 0:00:00
  Created wheel for liac-arff: filename=liac_arff-2.5.0-py3-none-any.whl size=11716 sha256=07f894e4f92433ffa2e1524a5c68f17011c189605e8d2ee34fa24e63046f9e6d
  Stored in directory: /root/.cache/pip/wheels/5d/2a/9c/3895d9617f8f49a0883ba686326d598e78a1c2f54fe3cae86d
Successfully built liac-arff


In [ ]:
pip install torch torchvision torchaudio


In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import openml
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Tabular Dataset class to handle data
class TabularDataset(Dataset):
    def __init__(self, features, targets, categorical_indices):
        self.features = torch.tensor(features.values, dtype=torch.float32)  # Convert to numpy array
        self.targets = torch.tensor(targets, dtype=torch.long)  # Convert to long for multiclass classification
        self.categorical_indices = categorical_indices

    def __len__(self):
        return len(self.targets)

    def __getitem__(self, idx):
        return self.features[idx], self.targets[idx]

# Load the data from OpenML
def load_data(task_id):
    task = openml.tasks.get_task(task_id)
    dataset = task.get_dataset()
    X, y, _, _ = dataset.get_data(target=dataset.default_target_attribute)

    cat_columns = [col for col in X.columns if X[col].dtype.name == 'category']
    num_columns = [col for col in X.columns if X[col].dtype in ['int64', 'float64']]

    return X, y, cat_columns, num_columns

def preprocess_data(X, cat_columns):
    # Handle missing values in categorical columns
    for col in cat_columns:
        X[col] = X[col].astype('category')  # Convert the column to 'category' type if not already
        X[col] = X[col].cat.add_categories('Unknown')  # Add 'Unknown' as a new category
        X[col] = X[col].fillna('Unknown')  # Replace NaN with 'Unknown'

    # Apply label encoding for categorical columns
    label_encoders = {}
    for col in cat_columns:
        le = LabelEncoder()
        X[col] = le.fit_transform(X[col].astype(str))  # Ensure string type before label encoding
        label_encoders[col] = le  # Store the encoder for future use

    # Convert all columns to float for PyTorch compatibility
    for col in X.columns:
        X[col] = X[col].astype(float)

    return X, label_encoders


# Define the SAINT model
class SAINT(nn.Module):
    def __init__(self, input_dim, params):
        super(SAINT, self).__init__()
        self.embedding_dim = params['dim']
        self.embedding = nn.Linear(input_dim, self.embedding_dim)
        self.fc = nn.Linear(self.embedding_dim, len(np.unique(params['target'])))  # Output for each class

    def forward(self, x):
        x = self.embedding(x)
        x = self.fc(x)
        return x

# Train the model
def train_model(X_train, y_train, X_val, y_val, params):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    train_dataset = TabularDataset(X_train, y_train, params['cat_idx'])
    val_dataset = TabularDataset(X_val, y_val, params['cat_idx'])
    train_loader = DataLoader(train_dataset, batch_size=params['batch_size'], shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=params['val_batch_size'], shuffle=False)

    model = SAINT(input_dim=X_train.shape[1], params=params).to(device)
    criterion = nn.CrossEntropyLoss()  # For multiclass classification
    optimizer = optim.AdamW(model.parameters(), lr=params['lr'])

    # Training loop
    for epoch in range(params['epochs']):
        model.train()
        for x_batch, y_batch in train_loader:
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)
            optimizer.zero_grad()
            outputs = model(x_batch)
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()

    # Validation loop
    model.eval()
    val_accuracy = 0
    with torch.no_grad():
        all_preds = []
        all_labels = []
        for x_batch, y_batch in val_loader:
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)
            outputs = model(x_batch)
            preds = torch.argmax(outputs, dim=1)  # Get class predictions for multiclass
            all_preds.append(preds.cpu().numpy())
            all_labels.append(y_batch.cpu().numpy())

        # Flatten the lists to compute accuracy
        all_preds = np.concatenate(all_preds)
        all_labels = np.concatenate(all_labels)

        val_accuracy = np.mean(all_preds == all_labels)  # Compute accuracy

    return model, val_accuracy

# Generate random hyperparameters
def get_random_parameters(seed):
    rs = np.random.RandomState(seed)
    params = {
        "dim": rs.choice([32, 64, 128, 256]),
        "depth": rs.choice([1, 2, 3, 6, 12]),
        "heads": rs.choice([2, 4, 8]),
        "dropout": rs.choice([0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8]),
        "epochs": 100,
        "batch_size": 64,
        "val_batch_size": 32,
        "lr": 0.00003,
        "cat_idx": [],
        "num_idx": [],
    }
    return params

# Task ID1: 14965

In [ ]:
# Load and preprocess the data
task_id = 14965
X, y, cat_columns, num_columns = load_data(task_id)
X, label_encoders = preprocess_data(X, cat_columns)

# Encode the target variable
target_encoder = LabelEncoder()
y_encoded = target_encoder.fit_transform(y)  # Encode the target labels to integers

params = get_random_parameters(seed=42)
params['cat_idx'] = [X.columns.get_loc(c) for c in cat_columns]
params['num_idx'] = [X.columns.get_loc(c) for c in num_columns if c in X.columns]
params['target'] = np.unique(y_encoded)  # Add the unique targets for the output layer size

# Split the data
X_train, X_val, y_train, y_val = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Train the model
model, accuracy = train_model(X_train, y_train, X_val, y_val, params)
print(f'Validation Accuracy: {accuracy:.4f}')


Validation Accuracy: 0.8853


# Task ID2: 9977

In [ ]:
# Load and preprocess the data
task_id = 9977
X, y, cat_columns, num_columns = load_data(task_id)
X, label_encoders = preprocess_data(X, cat_columns)

# Encode the target variable
target_encoder = LabelEncoder()
y_encoded = target_encoder.fit_transform(y)  # Encode the target labels to integers

params = get_random_parameters(seed=42)
params['cat_idx'] = [X.columns.get_loc(c) for c in cat_columns]
params['num_idx'] = [X.columns.get_loc(c) for c in num_columns if c in X.columns]
params['target'] = np.unique(y_encoded)  # Add the unique targets for the output layer size

# Split the data
X_train, X_val, y_train, y_val = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Train the model
model, accuracy = train_model(X_train, y_train, X_val, y_val, params)
print(f'Validation Accuracy: {accuracy:.4f}')


Validation Accuracy: 0.9450


# Task ID3: 34539

In [ ]:
# Load and preprocess the data
task_id = 34539
X, y, cat_columns, num_columns = load_data(task_id)
X, label_encoders = preprocess_data(X, cat_columns)

# Encode the target variable
target_encoder = LabelEncoder()
y_encoded = target_encoder.fit_transform(y)  # Encode the target labels to integers

params = get_random_parameters(seed=42)
params['cat_idx'] = [X.columns.get_loc(c) for c in cat_columns]
params['num_idx'] = [X.columns.get_loc(c) for c in num_columns if c in X.columns]
params['target'] = np.unique(y_encoded)  # Add the unique targets for the output layer size

# Split the data
X_train, X_val, y_train, y_val = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Train the model
model, accuracy = train_model(X_train, y_train, X_val, y_val, params)
print(f'Validation Accuracy: {accuracy:.4f}')


Validation Accuracy: 0.9437


# Task ID4: 146606

In [ ]:
# Load and preprocess the data
task_id = 146606
X, y, cat_columns, num_columns = load_data(task_id)
X, label_encoders = preprocess_data(X, cat_columns)

# Encode the target variable
target_encoder = LabelEncoder()
y_encoded = target_encoder.fit_transform(y)  # Encode the target labels to integers

params = get_random_parameters(seed=42)
params['cat_idx'] = [X.columns.get_loc(c) for c in cat_columns]
params['num_idx'] = [X.columns.get_loc(c) for c in num_columns if c in X.columns]
params['target'] = np.unique(y_encoded)  # Add the unique targets for the output layer size

# Split the data
X_train, X_val, y_train, y_val = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Train the model
model, accuracy = train_model(X_train, y_train, X_val, y_val, params)
print(f'Validation Accuracy: {accuracy:.4f}')


Validation Accuracy: 0.6380


# Task ID5: 7592



In [ ]:
# Load and preprocess the data
task_id = 7592
X, y, cat_columns, num_columns = load_data(task_id)
X, label_encoders = preprocess_data(X, cat_columns)

# Encode the target variable
target_encoder = LabelEncoder()
y_encoded = target_encoder.fit_transform(y)  # Encode the target labels to integers

params = get_random_parameters(seed=42)
params['cat_idx'] = [X.columns.get_loc(c) for c in cat_columns]
params['num_idx'] = [X.columns.get_loc(c) for c in num_columns if c in X.columns]
params['target'] = np.unique(y_encoded)  # Add the unique targets for the output layer size

# Split the data
X_train, X_val, y_train, y_val = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Train the model
model, accuracy = train_model(X_train, y_train, X_val, y_val, params)
print(f'Validation Accuracy: {accuracy:.4f}')


Validation Accuracy: 0.8008


# Task ID6: 146195



In [ ]:
# Load and preprocess the data
task_id = 146195
X, y, cat_columns, num_columns = load_data(task_id)
X, label_encoders = preprocess_data(X, cat_columns)

# Encode the target variable
target_encoder = LabelEncoder()
y_encoded = target_encoder.fit_transform(y)  # Encode the target labels to integers

params = get_random_parameters(seed=42)
params['cat_idx'] = [X.columns.get_loc(c) for c in cat_columns]
params['num_idx'] = [X.columns.get_loc(c) for c in num_columns if c in X.columns]
params['target'] = np.unique(y_encoded)  # Add the unique targets for the output layer size

# Split the data
X_train, X_val, y_train, y_val = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Train the model
model, accuracy = train_model(X_train, y_train, X_val, y_val, params)
print(f'Validation Accuracy: {accuracy:.4f}')


Validation Accuracy: 0.6584


# Task ID7: 167119



In [ ]:
# Load and preprocess the data
task_id = 167119
X, y, cat_columns, num_columns = load_data(task_id)
X, label_encoders = preprocess_data(X, cat_columns)

# Encode the target variable
target_encoder = LabelEncoder()
y_encoded = target_encoder.fit_transform(y)  # Encode the target labels to integers

params = get_random_parameters(seed=42)
params['cat_idx'] = [X.columns.get_loc(c) for c in cat_columns]
params['num_idx'] = [X.columns.get_loc(c) for c in num_columns if c in X.columns]
params['target'] = np.unique(y_encoded)  # Add the unique targets for the output layer size

# Split the data
X_train, X_val, y_train, y_val = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Train the model
model, accuracy = train_model(X_train, y_train, X_val, y_val, params)
print(f'Validation Accuracy: {accuracy:.4f}')


Validation Accuracy: 0.6746


# Task ID8: 167120



In [ ]:
# Load and preprocess the data
task_id = 167120
X, y, cat_columns, num_columns = load_data(task_id)
X, label_encoders = preprocess_data(X, cat_columns)

# Encode the target variable
target_encoder = LabelEncoder()
y_encoded = target_encoder.fit_transform(y)  # Encode the target labels to integers

params = get_random_parameters(seed=42)
params['cat_idx'] = [X.columns.get_loc(c) for c in cat_columns]
params['num_idx'] = [X.columns.get_loc(c) for c in num_columns if c in X.columns]
params['target'] = np.unique(y_encoded)  # Add the unique targets for the output layer size

# Split the data
X_train, X_val, y_train, y_val = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Train the model
model, accuracy = train_model(X_train, y_train, X_val, y_val, params)
print(f'Validation Accuracy: {accuracy:.4f}')


Validation Accuracy: 0.5197


# Task ID11: 	168331



In [ ]:
# Load and preprocess the data
task_id = 168331
X, y, cat_columns, num_columns = load_data(task_id)
X, label_encoders = preprocess_data(X, cat_columns)

# Encode the target variable
target_encoder = LabelEncoder()
y_encoded = target_encoder.fit_transform(y)  # Encode the target labels to integers

params = get_random_parameters(seed=42)
params['cat_idx'] = [X.columns.get_loc(c) for c in cat_columns]
params['num_idx'] = [X.columns.get_loc(c) for c in num_columns if c in X.columns]
params['target'] = np.unique(y_encoded)  # Add the unique targets for the output layer size

# Split the data
X_train, X_val, y_train, y_val = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Train the model
model, accuracy = train_model(X_train, y_train, X_val, y_val, params)
print(f'Validation Accuracy: {accuracy:.4f}')


Validation Accuracy: 0.5564


# Task ID12: 	168330



In [ ]:
# Load and preprocess the data
task_id = 168330
X, y, cat_columns, num_columns = load_data(task_id)
X, label_encoders = preprocess_data(X, cat_columns)

# Encode the target variable
target_encoder = LabelEncoder()
y_encoded = target_encoder.fit_transform(y)  # Encode the target labels to integers

params = get_random_parameters(seed=42)
params['cat_idx'] = [X.columns.get_loc(c) for c in cat_columns]
params['num_idx'] = [X.columns.get_loc(c) for c in num_columns if c in X.columns]
params['target'] = np.unique(y_encoded)  # Add the unique targets for the output layer size

# Split the data
X_train, X_val, y_train, y_val = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Train the model
model, accuracy = train_model(X_train, y_train, X_val, y_val, params)
print(f'Validation Accuracy: {accuracy:.4f}')


Validation Accuracy: 0.6357


# Task ID13: 	168335



In [ ]:
# Load and preprocess the data
task_id = 168335
X, y, cat_columns, num_columns = load_data(task_id)
X, label_encoders = preprocess_data(X, cat_columns)

# Encode the target variable
target_encoder = LabelEncoder()
y_encoded = target_encoder.fit_transform(y)  # Encode the target labels to integers

params = get_random_parameters(seed=42)
params['cat_idx'] = [X.columns.get_loc(c) for c in cat_columns]
params['num_idx'] = [X.columns.get_loc(c) for c in num_columns if c in X.columns]
params['target'] = np.unique(y_encoded)  # Add the unique targets for the output layer size

# Split the data
X_train, X_val, y_train, y_val = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Train the model
model, accuracy = train_model(X_train, y_train, X_val, y_val, params)
print(f'Validation Accuracy: {accuracy:.4f}')


Validation Accuracy: 0.8809



# Task ID16: 	146212



In [ ]:
# Load and preprocess the data
task_id = 146212
X, y, cat_columns, num_columns = load_data(task_id)
X, label_encoders = preprocess_data(X, cat_columns)

# Encode the target variable
target_encoder = LabelEncoder()
y_encoded = target_encoder.fit_transform(y)  # Encode the target labels to integers

params = get_random_parameters(seed=42)
params['cat_idx'] = [X.columns.get_loc(c) for c in cat_columns]
params['num_idx'] = [X.columns.get_loc(c) for c in num_columns if c in X.columns]
params['target'] = np.unique(y_encoded)  # Add the unique targets for the output layer size

# Split the data
X_train, X_val, y_train, y_val = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Train the model
model, accuracy = train_model(X_train, y_train, X_val, y_val, params)
print(f'Validation Accuracy: {accuracy:.4f}')


Validation Accuracy: 0.9280


# Task ID19: 	168868



In [ ]:
# Load and preprocess the data
task_id = 168868
X, y, cat_columns, num_columns = load_data(task_id)
X, label_encoders = preprocess_data(X, cat_columns)

# Encode the target variable
target_encoder = LabelEncoder()
y_encoded = target_encoder.fit_transform(y)  # Encode the target labels to integers

params = get_random_parameters(seed=42)
params['cat_idx'] = [X.columns.get_loc(c) for c in cat_columns]
params['num_idx'] = [X.columns.get_loc(c) for c in num_columns if c in X.columns]
params['target'] = np.unique(y_encoded)  # Add the unique targets for the output layer size

# Split the data
X_train, X_val, y_train, y_val = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Train the model
model, accuracy = train_model(X_train, y_train, X_val, y_val, params)
print(f'Validation Accuracy: {accuracy:.4f}')


Validation Accuracy: 0.9803


# Task ID20: 31

In [ ]:
# Load and preprocess the data
task_id = 31
X, y, cat_columns, num_columns = load_data(task_id)
X, label_encoders = preprocess_data(X, cat_columns)

# Encode the target variable
target_encoder = LabelEncoder()
y_encoded = target_encoder.fit_transform(y)  # Encode the target labels to integers

params = get_random_parameters(seed=42)
params['cat_idx'] = [X.columns.get_loc(c) for c in cat_columns]
params['num_idx'] = [X.columns.get_loc(c) for c in num_columns if c in X.columns]
params['target'] = np.unique(y_encoded)  # Add the unique targets for the output layer size

# Split the data
X_train, X_val, y_train, y_val = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Train the model
model, accuracy = train_model(X_train, y_train, X_val, y_val, params)
print(f'Validation Accuracy: {accuracy:.4f}')


Validation Accuracy: 0.7200


# Task ID21: 10101

In [ ]:
# Load and preprocess the data
task_id = 10101
X, y, cat_columns, num_columns = load_data(task_id)
X, label_encoders = preprocess_data(X, cat_columns)

# Encode the target variable
target_encoder = LabelEncoder()
y_encoded = target_encoder.fit_transform(y)  # Encode the target labels to integers

params = get_random_parameters(seed=42)
params['cat_idx'] = [X.columns.get_loc(c) for c in cat_columns]
params['num_idx'] = [X.columns.get_loc(c) for c in num_columns if c in X.columns]
params['target'] = np.unique(y_encoded)  # Add the unique targets for the output layer size

# Split the data
X_train, X_val, y_train, y_val = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Train the model
model, accuracy = train_model(X_train, y_train, X_val, y_val, params)
print(f'Validation Accuracy: {accuracy:.4f}')


Validation Accuracy: 0.6533


# Task ID22: 	3913

In [ ]:
# Load and preprocess the data
task_id = 3913
X, y, cat_columns, num_columns = load_data(task_id)
X, label_encoders = preprocess_data(X, cat_columns)

# Encode the target variable
target_encoder = LabelEncoder()
y_encoded = target_encoder.fit_transform(y)  # Encode the target labels to integers

params = get_random_parameters(seed=42)
params['cat_idx'] = [X.columns.get_loc(c) for c in cat_columns]
params['num_idx'] = [X.columns.get_loc(c) for c in num_columns if c in X.columns]
params['target'] = np.unique(y_encoded)  # Add the unique targets for the output layer size

# Split the data
X_train, X_val, y_train, y_val = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Train the model
model, accuracy = train_model(X_train, y_train, X_val, y_val, params)
print(f'Validation Accuracy: {accuracy:.4f}')


Validation Accuracy: 0.5714


# Task ID23: 3

In [ ]:
# Load and preprocess the data
task_id = 3
X, y, cat_columns, num_columns = load_data(task_id)
X, label_encoders = preprocess_data(X, cat_columns)

# Encode the target variable
target_encoder = LabelEncoder()
y_encoded = target_encoder.fit_transform(y)  # Encode the target labels to integers

params = get_random_parameters(seed=42)
params['cat_idx'] = [X.columns.get_loc(c) for c in cat_columns]
params['num_idx'] = [X.columns.get_loc(c) for c in num_columns if c in X.columns]
params['target'] = np.unique(y_encoded)  # Add the unique targets for the output layer size

# Split the data
X_train, X_val, y_train, y_val = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Train the model
model, accuracy = train_model(X_train, y_train, X_val, y_val, params)
print(f'Validation Accuracy: {accuracy:.4f}')


Validation Accuracy: 0.9141


# Task ID24: 3917

In [ ]:
# Load and preprocess the data
task_id = 3917
X, y, cat_columns, num_columns = load_data(task_id)
X, label_encoders = preprocess_data(X, cat_columns)

# Encode the target variable
target_encoder = LabelEncoder()
y_encoded = target_encoder.fit_transform(y)  # Encode the target labels to integers

params = get_random_parameters(seed=42)
params['cat_idx'] = [X.columns.get_loc(c) for c in cat_columns]
params['num_idx'] = [X.columns.get_loc(c) for c in num_columns if c in X.columns]
params['target'] = np.unique(y_encoded)  # Add the unique targets for the output layer size

# Split the data
X_train, X_val, y_train, y_val = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Train the model
model, accuracy = train_model(X_train, y_train, X_val, y_val, params)
print(f'Validation Accuracy: {accuracy:.4f}')


Validation Accuracy: 0.7796


# Task ID25: 9957

In [ ]:
# Load and preprocess the data
task_id = 9957
X, y, cat_columns, num_columns = load_data(task_id)
X, label_encoders = preprocess_data(X, cat_columns)

# Encode the target variable
target_encoder = LabelEncoder()
y_encoded = target_encoder.fit_transform(y)  # Encode the target labels to integers

params = get_random_parameters(seed=42)
params['cat_idx'] = [X.columns.get_loc(c) for c in cat_columns]
params['num_idx'] = [X.columns.get_loc(c) for c in num_columns if c in X.columns]
params['target'] = np.unique(y_encoded)  # Add the unique targets for the output layer size

# Split the data
X_train, X_val, y_train, y_val = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Train the model
model, accuracy = train_model(X_train, y_train, X_val, y_val, params)
print(f'Validation Accuracy: {accuracy:.4f}')


Validation Accuracy: 0.8294


# Task ID26: 9946

In [ ]:
# Load and preprocess the data
task_id = 9946
X, y, cat_columns, num_columns = load_data(task_id)
X, label_encoders = preprocess_data(X, cat_columns)

# Encode the target variable
target_encoder = LabelEncoder()
y_encoded = target_encoder.fit_transform(y)  # Encode the target labels to integers

params = get_random_parameters(seed=42)
params['cat_idx'] = [X.columns.get_loc(c) for c in cat_columns]
params['num_idx'] = [X.columns.get_loc(c) for c in num_columns if c in X.columns]
params['target'] = np.unique(y_encoded)  # Add the unique targets for the output layer size

# Split the data
X_train, X_val, y_train, y_val = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Train the model
model, accuracy = train_model(X_train, y_train, X_val, y_val, params)
print(f'Validation Accuracy: {accuracy:.4f}')


Validation Accuracy: 0.9298


# Task ID27: 3918

In [ ]:
# Load and preprocess the data
task_id = 3918
X, y, cat_columns, num_columns = load_data(task_id)
X, label_encoders = preprocess_data(X, cat_columns)

# Encode the target variable
target_encoder = LabelEncoder()
y_encoded = target_encoder.fit_transform(y)  # Encode the target labels to integers

params = get_random_parameters(seed=42)
params['cat_idx'] = [X.columns.get_loc(c) for c in cat_columns]
params['num_idx'] = [X.columns.get_loc(c) for c in num_columns if c in X.columns]
params['target'] = np.unique(y_encoded)  # Add the unique targets for the output layer size

# Split the data
X_train, X_val, y_train, y_val = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Train the model
model, accuracy = train_model(X_train, y_train, X_val, y_val, params)
print(f'Validation Accuracy: {accuracy:.4f}')


Validation Accuracy: 0.8108


# Task ID28: 3903

In [ ]:
# Load and preprocess the data
task_id = 3903
X, y, cat_columns, num_columns = load_data(task_id)
X, label_encoders = preprocess_data(X, cat_columns)

# Encode the target variable
target_encoder = LabelEncoder()
y_encoded = target_encoder.fit_transform(y)  # Encode the target labels to integers

params = get_random_parameters(seed=42)
params['cat_idx'] = [X.columns.get_loc(c) for c in cat_columns]
params['num_idx'] = [X.columns.get_loc(c) for c in num_columns if c in X.columns]
params['target'] = np.unique(y_encoded)  # Add the unique targets for the output layer size

# Split the data
X_train, X_val, y_train, y_val = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Train the model
model, accuracy = train_model(X_train, y_train, X_val, y_val, params)
print(f'Validation Accuracy: {accuracy:.4f}')


Validation Accuracy: 0.8051


# Task ID29: 37

In [ ]:
# Load and preprocess the data
task_id = 37
X, y, cat_columns, num_columns = load_data(task_id)
X, label_encoders = preprocess_data(X, cat_columns)

# Encode the target variable
target_encoder = LabelEncoder()
y_encoded = target_encoder.fit_transform(y)  # Encode the target labels to integers

params = get_random_parameters(seed=42)
params['cat_idx'] = [X.columns.get_loc(c) for c in cat_columns]
params['num_idx'] = [X.columns.get_loc(c) for c in num_columns if c in X.columns]
params['target'] = np.unique(y_encoded)  # Add the unique targets for the output layer size

# Split the data
X_train, X_val, y_train, y_val = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Train the model
model, accuracy = train_model(X_train, y_train, X_val, y_val, params)
print(f'Validation Accuracy: {accuracy:.4f}')


Validation Accuracy: 0.7013


# Task ID30: 9971

In [ ]:
# Load and preprocess the data
task_id = 9971
X, y, cat_columns, num_columns = load_data(task_id)
X, label_encoders = preprocess_data(X, cat_columns)

# Encode the target variable
target_encoder = LabelEncoder()
y_encoded = target_encoder.fit_transform(y)  # Encode the target labels to integers

params = get_random_parameters(seed=42)
params['cat_idx'] = [X.columns.get_loc(c) for c in cat_columns]
params['num_idx'] = [X.columns.get_loc(c) for c in num_columns if c in X.columns]
params['target'] = np.unique(y_encoded)  # Add the unique targets for the output layer size

# Split the data
X_train, X_val, y_train, y_val = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Train the model
model, accuracy = train_model(X_train, y_train, X_val, y_val, params)
print(f'Validation Accuracy: {accuracy:.4f}')


Validation Accuracy: 0.7436


# Task ID31: 9952

In [ ]:
# Load and preprocess the data
task_id = 9952
X, y, cat_columns, num_columns = load_data(task_id)
X, label_encoders = preprocess_data(X, cat_columns)

# Encode the target variable
target_encoder = LabelEncoder()
y_encoded = target_encoder.fit_transform(y)  # Encode the target labels to integers

params = get_random_parameters(seed=42)
params['cat_idx'] = [X.columns.get_loc(c) for c in cat_columns]
params['num_idx'] = [X.columns.get_loc(c) for c in num_columns if c in X.columns]
params['target'] = np.unique(y_encoded)  # Add the unique targets for the output layer size

# Split the data
X_train, X_val, y_train, y_val = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Train the model
model, accuracy = train_model(X_train, y_train, X_val, y_val, params)
print(f'Validation Accuracy: {accuracy:.4f}')


Validation Accuracy: 0.7576


# Task ID32: 3902

In [ ]:
# Load and preprocess the data
task_id = 3902
X, y, cat_columns, num_columns = load_data(task_id)
X, label_encoders = preprocess_data(X, cat_columns)

# Encode the target variable
target_encoder = LabelEncoder()
y_encoded = target_encoder.fit_transform(y)  # Encode the target labels to integers

params = get_random_parameters(seed=42)
params['cat_idx'] = [X.columns.get_loc(c) for c in cat_columns]
params['num_idx'] = [X.columns.get_loc(c) for c in num_columns if c in X.columns]
params['target'] = np.unique(y_encoded)  # Add the unique targets for the output layer size

# Split the data
X_train, X_val, y_train, y_val = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Train the model
model, accuracy = train_model(X_train, y_train, X_val, y_val, params)
print(f'Validation Accuracy: {accuracy:.4f}')


Validation Accuracy: 0.6644


# Task ID33: 49

In [ ]:
# Load and preprocess the data
task_id = 49
X, y, cat_columns, num_columns = load_data(task_id)
X, label_encoders = preprocess_data(X, cat_columns)

# Encode the target variable
target_encoder = LabelEncoder()
y_encoded = target_encoder.fit_transform(y)  # Encode the target labels to integers

params = get_random_parameters(seed=42)
params['cat_idx'] = [X.columns.get_loc(c) for c in cat_columns]
params['num_idx'] = [X.columns.get_loc(c) for c in num_columns if c in X.columns]
params['target'] = np.unique(y_encoded)  # Add the unique targets for the output layer size

# Split the data
X_train, X_val, y_train, y_val = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Train the model
model, accuracy = train_model(X_train, y_train, X_val, y_val, params)
print(f'Validation Accuracy: {accuracy:.4f}')


Validation Accuracy: 0.6667


# Task ID34: 43

In [ ]:
# Load and preprocess the data
task_id = 43
X, y, cat_columns, num_columns = load_data(task_id)
X, label_encoders = preprocess_data(X, cat_columns)

# Encode the target variable
target_encoder = LabelEncoder()
y_encoded = target_encoder.fit_transform(y)  # Encode the target labels to integers

params = get_random_parameters(seed=42)
params['cat_idx'] = [X.columns.get_loc(c) for c in cat_columns]
params['num_idx'] = [X.columns.get_loc(c) for c in num_columns if c in X.columns]
params['target'] = np.unique(y_encoded)  # Add the unique targets for the output layer size

# Split the data
X_train, X_val, y_train, y_val = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Train the model
model, accuracy = train_model(X_train, y_train, X_val, y_val, params)
print(f'Validation Accuracy: {accuracy:.4f}')


Validation Accuracy: 0.9229


# Task ID35: 9978

In [ ]:
# Load and preprocess the data
task_id = 9978
X, y, cat_columns, num_columns = load_data(task_id)
X, label_encoders = preprocess_data(X, cat_columns)

# Encode the target variable
target_encoder = LabelEncoder()
y_encoded = target_encoder.fit_transform(y)  # Encode the target labels to integers

params = get_random_parameters(seed=42)
params['cat_idx'] = [X.columns.get_loc(c) for c in cat_columns]
params['num_idx'] = [X.columns.get_loc(c) for c in num_columns if c in X.columns]
params['target'] = np.unique(y_encoded)  # Add the unique targets for the output layer size

# Split the data
X_train, X_val, y_train, y_val = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Train the model
model, accuracy = train_model(X_train, y_train, X_val, y_val, params)
print(f'Validation Accuracy: {accuracy:.4f}')


Validation Accuracy: 0.9349


# Task ID36: 10093

In [ ]:
# Load and preprocess the data
task_id = 10093
X, y, cat_columns, num_columns = load_data(task_id)
X, label_encoders = preprocess_data(X, cat_columns)

# Encode the target variable
target_encoder = LabelEncoder()
y_encoded = target_encoder.fit_transform(y)  # Encode the target labels to integers

params = get_random_parameters(seed=42)
params['cat_idx'] = [X.columns.get_loc(c) for c in cat_columns]
params['num_idx'] = [X.columns.get_loc(c) for c in num_columns if c in X.columns]
params['target'] = np.unique(y_encoded)  # Add the unique targets for the output layer size

# Split the data
X_train, X_val, y_train, y_val = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Train the model
model, accuracy = train_model(X_train, y_train, X_val, y_val, params)
print(f'Validation Accuracy: {accuracy:.4f}')


Validation Accuracy: 0.9673


# Task ID37: 219

In [ ]:
# Load and preprocess the data
task_id = 219
X, y, cat_columns, num_columns = load_data(task_id)
X, label_encoders = preprocess_data(X, cat_columns)

# Encode the target variable
target_encoder = LabelEncoder()
y_encoded = target_encoder.fit_transform(y)  # Encode the target labels to integers

params = get_random_parameters(seed=42)
params['cat_idx'] = [X.columns.get_loc(c) for c in cat_columns]
params['num_idx'] = [X.columns.get_loc(c) for c in num_columns if c in X.columns]
params['target'] = np.unique(y_encoded)  # Add the unique targets for the output layer size

# Split the data
X_train, X_val, y_train, y_val = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Train the model
model, accuracy = train_model(X_train, y_train, X_val, y_val, params)
print(f'Validation Accuracy: {accuracy:.4f}')


Validation Accuracy: 0.7499


# Task ID38: 9976

In [ ]:
# Load and preprocess the data
task_id = 9976
X, y, cat_columns, num_columns = load_data(task_id)
X, label_encoders = preprocess_data(X, cat_columns)

# Encode the target variable
target_encoder = LabelEncoder()
y_encoded = target_encoder.fit_transform(y)  # Encode the target labels to integers

params = get_random_parameters(seed=42)
params['cat_idx'] = [X.columns.get_loc(c) for c in cat_columns]
params['num_idx'] = [X.columns.get_loc(c) for c in num_columns if c in X.columns]
params['target'] = np.unique(y_encoded)  # Add the unique targets for the output layer size

# Split the data
X_train, X_val, y_train, y_val = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Train the model
model, accuracy = train_model(X_train, y_train, X_val, y_val, params)
print(f'Validation Accuracy: {accuracy:.4f}')


Validation Accuracy: 0.5231


# Task ID39: 6

In [ ]:
# Load and preprocess the data
task_id = 6
X, y, cat_columns, num_columns = load_data(task_id)
X, label_encoders = preprocess_data(X, cat_columns)

# Encode the target variable
target_encoder = LabelEncoder()
y_encoded = target_encoder.fit_transform(y)  # Encode the target labels to integers

params = get_random_parameters(seed=42)
params['cat_idx'] = [X.columns.get_loc(c) for c in cat_columns]
params['num_idx'] = [X.columns.get_loc(c) for c in num_columns if c in X.columns]
params['target'] = np.unique(y_encoded)  # Add the unique targets for the output layer size

# Split the data
X_train, X_val, y_train, y_val = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Train the model
model, accuracy = train_model(X_train, y_train, X_val, y_val, params)
print(f'Validation Accuracy: {accuracy:.4f}')


Validation Accuracy: 0.7515


# Task ID40: 53

In [ ]:
# Load and preprocess the data
task_id = 53
X, y, cat_columns, num_columns = load_data(task_id)
X, label_encoders = preprocess_data(X, cat_columns)

# Encode the target variable
target_encoder = LabelEncoder()
y_encoded = target_encoder.fit_transform(y)  # Encode the target labels to integers

params = get_random_parameters(seed=42)
params['cat_idx'] = [X.columns.get_loc(c) for c in cat_columns]
params['num_idx'] = [X.columns.get_loc(c) for c in num_columns if c in X.columns]
params['target'] = np.unique(y_encoded)  # Add the unique targets for the output layer size

# Split the data
X_train, X_val, y_train, y_val = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Train the model
model, accuracy = train_model(X_train, y_train, X_val, y_val, params)
print(f'Validation Accuracy: {accuracy:.4f}')


Validation Accuracy: 0.3706


# Task ID41: 11

In [ ]:
# Load and preprocess the data
task_id = 11
X, y, cat_columns, num_columns = load_data(task_id)
X, label_encoders = preprocess_data(X, cat_columns)

# Encode the target variable
target_encoder = LabelEncoder()
y_encoded = target_encoder.fit_transform(y)  # Encode the target labels to integers

params = get_random_parameters(seed=42)
params['cat_idx'] = [X.columns.get_loc(c) for c in cat_columns]
params['num_idx'] = [X.columns.get_loc(c) for c in num_columns if c in X.columns]
params['target'] = np.unique(y_encoded)  # Add the unique targets for the output layer size

# Split the data
X_train, X_val, y_train, y_val = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Train the model
model, accuracy = train_model(X_train, y_train, X_val, y_val, params)
print(f'Validation Accuracy: {accuracy:.4f}')


Validation Accuracy: 0.8400


# Task ID42: 15

In [ ]:
# Load and preprocess the data
task_id = 15
X, y, cat_columns, num_columns = load_data(task_id)
X, label_encoders = preprocess_data(X, cat_columns)

# Encode the target variable
target_encoder = LabelEncoder()
y_encoded = target_encoder.fit_transform(y)  # Encode the target labels to integers

params = get_random_parameters(seed=42)
params['cat_idx'] = [X.columns.get_loc(c) for c in cat_columns]
params['num_idx'] = [X.columns.get_loc(c) for c in num_columns if c in X.columns]
params['target'] = np.unique(y_encoded)  # Add the unique targets for the output layer size

# Split the data
X_train, X_val, y_train, y_val = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Train the model
model, accuracy = train_model(X_train, y_train, X_val, y_val, params)
print(f'Validation Accuracy: {accuracy:.4f}')


Validation Accuracy: 0.6786


# Task ID43: 16

In [ ]:
# Load and preprocess the data
task_id = 16
X, y, cat_columns, num_columns = load_data(task_id)
X, label_encoders = preprocess_data(X, cat_columns)

# Encode the target variable
target_encoder = LabelEncoder()
y_encoded = target_encoder.fit_transform(y)  # Encode the target labels to integers

params = get_random_parameters(seed=42)
params['cat_idx'] = [X.columns.get_loc(c) for c in cat_columns]
params['num_idx'] = [X.columns.get_loc(c) for c in num_columns if c in X.columns]
params['target'] = np.unique(y_encoded)  # Add the unique targets for the output layer size

# Split the data
X_train, X_val, y_train, y_val = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Train the model
model, accuracy = train_model(X_train, y_train, X_val, y_val, params)
print(f'Validation Accuracy: {accuracy:.4f}')


Validation Accuracy: 0.9600


# Task ID44: 14

In [ ]:
# Load and preprocess the data
task_id = 14
X, y, cat_columns, num_columns = load_data(task_id)
X, label_encoders = preprocess_data(X, cat_columns)

# Encode the target variable
target_encoder = LabelEncoder()
y_encoded = target_encoder.fit_transform(y)  # Encode the target labels to integers

params = get_random_parameters(seed=42)
params['cat_idx'] = [X.columns.get_loc(c) for c in cat_columns]
params['num_idx'] = [X.columns.get_loc(c) for c in num_columns if c in X.columns]
params['target'] = np.unique(y_encoded)  # Add the unique targets for the output layer size

# Split the data
X_train, X_val, y_train, y_val = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Train the model
model, accuracy = train_model(X_train, y_train, X_val, y_val, params)
print(f'Validation Accuracy: {accuracy:.4f}')


Validation Accuracy: 0.5575


# Task ID45: 32

In [ ]:
# Load and preprocess the data
task_id = 32
X, y, cat_columns, num_columns = load_data(task_id)
X, label_encoders = preprocess_data(X, cat_columns)

# Encode the target variable
target_encoder = LabelEncoder()
y_encoded = target_encoder.fit_transform(y)  # Encode the target labels to integers

params = get_random_parameters(seed=42)
params['cat_idx'] = [X.columns.get_loc(c) for c in cat_columns]
params['num_idx'] = [X.columns.get_loc(c) for c in num_columns if c in X.columns]
params['target'] = np.unique(y_encoded)  # Add the unique targets for the output layer size

# Split the data
X_train, X_val, y_train, y_val = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Train the model
model, accuracy = train_model(X_train, y_train, X_val, y_val, params)
print(f'Validation Accuracy: {accuracy:.4f}')


Validation Accuracy: 0.9363


# Task ID46: 3549

In [ ]:
# Load and preprocess the data
task_id = 3549
X, y, cat_columns, num_columns = load_data(task_id)
X, label_encoders = preprocess_data(X, cat_columns)

# Encode the target variable
target_encoder = LabelEncoder()
y_encoded = target_encoder.fit_transform(y)  # Encode the target labels to integers

params = get_random_parameters(seed=42)
params['cat_idx'] = [X.columns.get_loc(c) for c in cat_columns]
params['num_idx'] = [X.columns.get_loc(c) for c in num_columns if c in X.columns]
params['target'] = np.unique(y_encoded)  # Add the unique targets for the output layer size

# Split the data
X_train, X_val, y_train, y_val = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Train the model
model, accuracy = train_model(X_train, y_train, X_val, y_val, params)
print(f'Validation Accuracy: {accuracy:.4f}')


Validation Accuracy: 0.9941


# Task ID47: 12

In [ ]:
# Load and preprocess the data
task_id = 12
X, y, cat_columns, num_columns = load_data(task_id)
X, label_encoders = preprocess_data(X, cat_columns)

# Encode the target variable
target_encoder = LabelEncoder()
y_encoded = target_encoder.fit_transform(y)  # Encode the target labels to integers

params = get_random_parameters(seed=42)
params['cat_idx'] = [X.columns.get_loc(c) for c in cat_columns]
params['num_idx'] = [X.columns.get_loc(c) for c in num_columns if c in X.columns]
params['target'] = np.unique(y_encoded)  # Add the unique targets for the output layer size

# Split the data
X_train, X_val, y_train, y_val = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Train the model
model, accuracy = train_model(X_train, y_train, X_val, y_val, params)
print(f'Validation Accuracy: {accuracy:.4f}')


Validation Accuracy: 0.9000


# Task ID48: 9981

In [ ]:
# Load and preprocess the data
task_id = 9981
X, y, cat_columns, num_columns = load_data(task_id)
X, label_encoders = preprocess_data(X, cat_columns)

# Encode the target variable
target_encoder = LabelEncoder()
y_encoded = target_encoder.fit_transform(y)  # Encode the target labels to integers

params = get_random_parameters(seed=42)
params['cat_idx'] = [X.columns.get_loc(c) for c in cat_columns]
params['num_idx'] = [X.columns.get_loc(c) for c in num_columns if c in X.columns]
params['target'] = np.unique(y_encoded)  # Add the unique targets for the output layer size

# Split the data
X_train, X_val, y_train, y_val = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Train the model
model, accuracy = train_model(X_train, y_train, X_val, y_val, params)
print(f'Validation Accuracy: {accuracy:.4f}')


Validation Accuracy: 0.9444


# Task ID49: 18

In [ ]:
# Load and preprocess the data
task_id = 18
X, y, cat_columns, num_columns = load_data(task_id)
X, label_encoders = preprocess_data(X, cat_columns)

# Encode the target variable
target_encoder = LabelEncoder()
y_encoded = target_encoder.fit_transform(y)  # Encode the target labels to integers

params = get_random_parameters(seed=42)
params['cat_idx'] = [X.columns.get_loc(c) for c in cat_columns]
params['num_idx'] = [X.columns.get_loc(c) for c in num_columns if c in X.columns]
params['target'] = np.unique(y_encoded)  # Add the unique targets for the output layer size

# Split the data
X_train, X_val, y_train, y_val = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Train the model
model, accuracy = train_model(X_train, y_train, X_val, y_val, params)
print(f'Validation Accuracy: {accuracy:.4f}')


Validation Accuracy: 0.4325


# Task ID50: 28

In [ ]:
# Load and preprocess the data
task_id = 28
X, y, cat_columns, num_columns = load_data(task_id)
X, label_encoders = preprocess_data(X, cat_columns)

# Encode the target variable
target_encoder = LabelEncoder()
y_encoded = target_encoder.fit_transform(y)  # Encode the target labels to integers

params = get_random_parameters(seed=42)
params['cat_idx'] = [X.columns.get_loc(c) for c in cat_columns]
params['num_idx'] = [X.columns.get_loc(c) for c in num_columns if c in X.columns]
params['target'] = np.unique(y_encoded)  # Add the unique targets for the output layer size

# Split the data
X_train, X_val, y_train, y_val = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Train the model
model, accuracy = train_model(X_train, y_train, X_val, y_val, params)
print(f'Validation Accuracy: {accuracy:.4f}')


Validation Accuracy: 0.9600


# Task ID51: 2074

In [ ]:
# Load and preprocess the data
task_id = 2074
X, y, cat_columns, num_columns = load_data(task_id)
X, label_encoders = preprocess_data(X, cat_columns)

# Encode the target variable
target_encoder = LabelEncoder()
y_encoded = target_encoder.fit_transform(y)  # Encode the target labels to integers

params = get_random_parameters(seed=42)
params['cat_idx'] = [X.columns.get_loc(c) for c in cat_columns]
params['num_idx'] = [X.columns.get_loc(c) for c in num_columns if c in X.columns]
params['target'] = np.unique(y_encoded)  # Add the unique targets for the output layer size

# Split the data
X_train, X_val, y_train, y_val = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Train the model
model, accuracy = train_model(X_train, y_train, X_val, y_val, params)
print(f'Validation Accuracy: {accuracy:.4f}')


Validation Accuracy: 0.8375


# Task ID52: 29

In [ ]:
# Load and preprocess the data
task_id = 29
X, y, cat_columns, num_columns = load_data(task_id)
X, label_encoders = preprocess_data(X, cat_columns)

# Encode the target variable
target_encoder = LabelEncoder()
y_encoded = target_encoder.fit_transform(y)  # Encode the target labels to integers

params = get_random_parameters(seed=42)
params['cat_idx'] = [X.columns.get_loc(c) for c in cat_columns]
params['num_idx'] = [X.columns.get_loc(c) for c in num_columns if c in X.columns]
params['target'] = np.unique(y_encoded)  # Add the unique targets for the output layer size

# Split the data
X_train, X_val, y_train, y_val = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Train the model
model, accuracy = train_model(X_train, y_train, X_val, y_val, params)
print(f'Validation Accuracy: {accuracy:.4f}')


Validation Accuracy: 0.5072


# Task ID53: 45

In [ ]:
# Load and preprocess the data
task_id = 45
X, y, cat_columns, num_columns = load_data(task_id)
X, label_encoders = preprocess_data(X, cat_columns)

# Encode the target variable
target_encoder = LabelEncoder()
y_encoded = target_encoder.fit_transform(y)  # Encode the target labels to integers

params = get_random_parameters(seed=42)
params['cat_idx'] = [X.columns.get_loc(c) for c in cat_columns]
params['num_idx'] = [X.columns.get_loc(c) for c in num_columns if c in X.columns]
params['target'] = np.unique(y_encoded)  # Add the unique targets for the output layer size

# Split the data
X_train, X_val, y_train, y_val = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Train the model
model, accuracy = train_model(X_train, y_train, X_val, y_val, params)
print(f'Validation Accuracy: {accuracy:.4f}')


Validation Accuracy: 0.7994


# Task ID54: 125922

In [ ]:
# Load and preprocess the data
task_id = 125922
X, y, cat_columns, num_columns = load_data(task_id)
X, label_encoders = preprocess_data(X, cat_columns)

# Encode the target variable
target_encoder = LabelEncoder()
y_encoded = target_encoder.fit_transform(y)  # Encode the target labels to integers

params = get_random_parameters(seed=42)
params['cat_idx'] = [X.columns.get_loc(c) for c in cat_columns]
params['num_idx'] = [X.columns.get_loc(c) for c in num_columns if c in X.columns]
params['target'] = np.unique(y_encoded)  # Add the unique targets for the output layer size

# Split the data
X_train, X_val, y_train, y_val = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Train the model
model, accuracy = train_model(X_train, y_train, X_val, y_val, params)
print(f'Validation Accuracy: {accuracy:.4f}')


Validation Accuracy: 0.8882


# Task ID55: 9960

In [ ]:
# Load and preprocess the data
task_id = 9960
X, y, cat_columns, num_columns = load_data(task_id)
X, label_encoders = preprocess_data(X, cat_columns)

# Encode the target variable
target_encoder = LabelEncoder()
y_encoded = target_encoder.fit_transform(y)  # Encode the target labels to integers

params = get_random_parameters(seed=42)
params['cat_idx'] = [X.columns.get_loc(c) for c in cat_columns]
params['num_idx'] = [X.columns.get_loc(c) for c in num_columns if c in X.columns]
params['target'] = np.unique(y_encoded)  # Add the unique targets for the output layer size

# Split the data
X_train, X_val, y_train, y_val = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Train the model
model, accuracy = train_model(X_train, y_train, X_val, y_val, params)
print(f'Validation Accuracy: {accuracy:.4f}')


Validation Accuracy: 0.6822


# Task ID56: 9964

In [ ]:
# Load and preprocess the data
task_id = 9964
X, y, cat_columns, num_columns = load_data(task_id)
X, label_encoders = preprocess_data(X, cat_columns)

# Encode the target variable
target_encoder = LabelEncoder()
y_encoded = target_encoder.fit_transform(y)  # Encode the target labels to integers

params = get_random_parameters(seed=42)
params['cat_idx'] = [X.columns.get_loc(c) for c in cat_columns]
params['num_idx'] = [X.columns.get_loc(c) for c in num_columns if c in X.columns]
params['target'] = np.unique(y_encoded)  # Add the unique targets for the output layer size

# Split the data
X_train, X_val, y_train, y_val = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Train the model
model, accuracy = train_model(X_train, y_train, X_val, y_val, params)
print(f'Validation Accuracy: {accuracy:.4f}')


Validation Accuracy: 0.8777


# Task ID57: 22

In [ ]:
# Load and preprocess the data
task_id = 22
X, y, cat_columns, num_columns = load_data(task_id)
X, label_encoders = preprocess_data(X, cat_columns)

# Encode the target variable
target_encoder = LabelEncoder()
y_encoded = target_encoder.fit_transform(y)  # Encode the target labels to integers

params = get_random_parameters(seed=42)
params['cat_idx'] = [X.columns.get_loc(c) for c in cat_columns]
params['num_idx'] = [X.columns.get_loc(c) for c in num_columns if c in X.columns]
params['target'] = np.unique(y_encoded)  # Add the unique targets for the output layer size

# Split the data
X_train, X_val, y_train, y_val = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Train the model
model, accuracy = train_model(X_train, y_train, X_val, y_val, params)
print(f'Validation Accuracy: {accuracy:.4f}')


Validation Accuracy: 0.7800


# Task ID58: 2079

In [ ]:
# Load and preprocess the data
task_id = 2079
X, y, cat_columns, num_columns = load_data(task_id)
X, label_encoders = preprocess_data(X, cat_columns)

# Encode the target variable
target_encoder = LabelEncoder()
y_encoded = target_encoder.fit_transform(y)  # Encode the target labels to integers

params = get_random_parameters(seed=42)
params['cat_idx'] = [X.columns.get_loc(c) for c in cat_columns]
params['num_idx'] = [X.columns.get_loc(c) for c in num_columns if c in X.columns]
params['target'] = np.unique(y_encoded)  # Add the unique targets for the output layer size

# Split the data
X_train, X_val, y_train, y_val = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Train the model
model, accuracy = train_model(X_train, y_train, X_val, y_val, params)
print(f'Validation Accuracy: {accuracy:.4f}')


Validation Accuracy: 0.1486


# Task ID59: 14969

In [ ]:
# Load and preprocess the data
task_id = 14969
X, y, cat_columns, num_columns = load_data(task_id)
X, label_encoders = preprocess_data(X, cat_columns)

# Encode the target variable
target_encoder = LabelEncoder()
y_encoded = target_encoder.fit_transform(y)  # Encode the target labels to integers

params = get_random_parameters(seed=42)
params['cat_idx'] = [X.columns.get_loc(c) for c in cat_columns]
params['num_idx'] = [X.columns.get_loc(c) for c in num_columns if c in X.columns]
params['target'] = np.unique(y_encoded)  # Add the unique targets for the output layer size

# Split the data
X_train, X_val, y_train, y_val = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Train the model
model, accuracy = train_model(X_train, y_train, X_val, y_val, params)
print(f'Validation Accuracy: {accuracy:.4f}')


Validation Accuracy: 0.4385


# Task ID60: 3560

In [ ]:
# Load and preprocess the data
task_id = 3560
X, y, cat_columns, num_columns = load_data(task_id)
X, label_encoders = preprocess_data(X, cat_columns)

# Encode the target variable
target_encoder = LabelEncoder()
y_encoded = target_encoder.fit_transform(y)  # Encode the target labels to integers

params = get_random_parameters(seed=42)
params['cat_idx'] = [X.columns.get_loc(c) for c in cat_columns]
params['num_idx'] = [X.columns.get_loc(c) for c in num_columns if c in X.columns]
params['target'] = np.unique(y_encoded)  # Add the unique targets for the output layer size

# Split the data
X_train, X_val, y_train, y_val = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Train the model
model, accuracy = train_model(X_train, y_train, X_val, y_val, params)
print(f'Validation Accuracy: {accuracy:.4f}')


Validation Accuracy: 0.1500


# Task ID61: 14952

In [ ]:
# Load and preprocess the data
task_id = 14952
X, y, cat_columns, num_columns = load_data(task_id)
X, label_encoders = preprocess_data(X, cat_columns)

# Encode the target variable
target_encoder = LabelEncoder()
y_encoded = target_encoder.fit_transform(y)  # Encode the target labels to integers

params = get_random_parameters(seed=42)
params['cat_idx'] = [X.columns.get_loc(c) for c in cat_columns]
params['num_idx'] = [X.columns.get_loc(c) for c in num_columns if c in X.columns]
params['target'] = np.unique(y_encoded)  # Add the unique targets for the output layer size

# Split the data
X_train, X_val, y_train, y_val = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Train the model
model, accuracy = train_model(X_train, y_train, X_val, y_val, params)
print(f'Validation Accuracy: {accuracy:.4f}')


Validation Accuracy: 0.9172


# Task ID62: 125920

In [ ]:
# Load and preprocess the data
task_id = 125920
X, y, cat_columns, num_columns = load_data(task_id)
X, label_encoders = preprocess_data(X, cat_columns)

# Encode the target variable
target_encoder = LabelEncoder()
y_encoded = target_encoder.fit_transform(y)  # Encode the target labels to integers

params = get_random_parameters(seed=42)
params['cat_idx'] = [X.columns.get_loc(c) for c in cat_columns]
params['num_idx'] = [X.columns.get_loc(c) for c in num_columns if c in X.columns]
params['target'] = np.unique(y_encoded)  # Add the unique targets for the output layer size

# Split the data
X_train, X_val, y_train, y_val = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Train the model
model, accuracy = train_model(X_train, y_train, X_val, y_val, params)
print(f'Validation Accuracy: {accuracy:.4f}')


Validation Accuracy: 0.6000


# Task ID63: 23

In [ ]:
# Load and preprocess the data
task_id = 23
X, y, cat_columns, num_columns = load_data(task_id)
X, label_encoders = preprocess_data(X, cat_columns)

# Encode the target variable
target_encoder = LabelEncoder()
y_encoded = target_encoder.fit_transform(y)  # Encode the target labels to integers

params = get_random_parameters(seed=42)
params['cat_idx'] = [X.columns.get_loc(c) for c in cat_columns]
params['num_idx'] = [X.columns.get_loc(c) for c in num_columns if c in X.columns]
params['target'] = np.unique(y_encoded)  # Add the unique targets for the output layer size

# Split the data
X_train, X_val, y_train, y_val = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Train the model
model, accuracy = train_model(X_train, y_train, X_val, y_val, params)
print(f'Validation Accuracy: {accuracy:.4f}')


Validation Accuracy: 0.4847


# Task ID64: 3904

In [ ]:
# Load and preprocess the data
task_id = 3904
X, y, cat_columns, num_columns = load_data(task_id)
X, label_encoders = preprocess_data(X, cat_columns)

# Encode the target variable
target_encoder = LabelEncoder()
y_encoded = target_encoder.fit_transform(y)  # Encode the target labels to integers

params = get_random_parameters(seed=42)
params['cat_idx'] = [X.columns.get_loc(c) for c in cat_columns]
params['num_idx'] = [X.columns.get_loc(c) for c in num_columns if c in X.columns]
params['target'] = np.unique(y_encoded)  # Add the unique targets for the output layer size

# Split the data
X_train, X_val, y_train, y_val = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Train the model
model, accuracy = train_model(X_train, y_train, X_val, y_val, params)
print(f'Validation Accuracy: {accuracy:.4f}')


Validation Accuracy: 0.8075


# Task ID65: 3022

In [ ]:
# Load and preprocess the data
task_id = 3022
X, y, cat_columns, num_columns = load_data(task_id)
X, label_encoders = preprocess_data(X, cat_columns)

# Encode the target variable
target_encoder = LabelEncoder()
y_encoded = target_encoder.fit_transform(y)  # Encode the target labels to integers

params = get_random_parameters(seed=42)
params['cat_idx'] = [X.columns.get_loc(c) for c in cat_columns]
params['num_idx'] = [X.columns.get_loc(c) for c in num_columns if c in X.columns]
params['target'] = np.unique(y_encoded)  # Add the unique targets for the output layer size

# Split the data
X_train, X_val, y_train, y_val = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Train the model
model, accuracy = train_model(X_train, y_train, X_val, y_val, params)
print(f'Validation Accuracy: {accuracy:.4f}')


Validation Accuracy: 0.2323


# Task ID66: 9985

In [ ]:
# Load and preprocess the data
task_id = 9985
X, y, cat_columns, num_columns = load_data(task_id)
X, label_encoders = preprocess_data(X, cat_columns)

# Encode the target variable
target_encoder = LabelEncoder()
y_encoded = target_encoder.fit_transform(y)  # Encode the target labels to integers

params = get_random_parameters(seed=42)
params['cat_idx'] = [X.columns.get_loc(c) for c in cat_columns]
params['num_idx'] = [X.columns.get_loc(c) for c in num_columns if c in X.columns]
params['target'] = np.unique(y_encoded)  # Add the unique targets for the output layer size

# Split the data
X_train, X_val, y_train, y_val = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Train the model
model, accuracy = train_model(X_train, y_train, X_val, y_val, params)
print(f'Validation Accuracy: {accuracy:.4f}')


Validation Accuracy: 0.4436


# Task ID67: 9910

In [ ]:
# Load and preprocess the data
task_id = 9910
X, y, cat_columns, num_columns = load_data(task_id)
X, label_encoders = preprocess_data(X, cat_columns)

# Encode the target variable
target_encoder = LabelEncoder()
y_encoded = target_encoder.fit_transform(y)  # Encode the target labels to integers

params = get_random_parameters(seed=42)
params['cat_idx'] = [X.columns.get_loc(c) for c in cat_columns]
params['num_idx'] = [X.columns.get_loc(c) for c in num_columns if c in X.columns]
params['target'] = np.unique(y_encoded)  # Add the unique targets for the output layer size

# Split the data
X_train, X_val, y_train, y_val = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Train the model
model, accuracy = train_model(X_train, y_train, X_val, y_val, params)
print(f'Validation Accuracy: {accuracy:.4f}')


Validation Accuracy: 0.7630


# Task ID68: 14970

In [ ]:
# Load and preprocess the data
task_id = 14970
X, y, cat_columns, num_columns = load_data(task_id)
X, label_encoders = preprocess_data(X, cat_columns)

# Encode the target variable
target_encoder = LabelEncoder()
y_encoded = target_encoder.fit_transform(y)  # Encode the target labels to integers

params = get_random_parameters(seed=42)
params['cat_idx'] = [X.columns.get_loc(c) for c in cat_columns]
params['num_idx'] = [X.columns.get_loc(c) for c in num_columns if c in X.columns]
params['target'] = np.unique(y_encoded)  # Add the unique targets for the output layer size

# Split the data
X_train, X_val, y_train, y_val = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Train the model
model, accuracy = train_model(X_train, y_train, X_val, y_val, params)
print(f'Validation Accuracy: {accuracy:.4f}')


Validation Accuracy: 0.9738


# Task ID69: 3021

In [ ]:
# Load and preprocess the data
task_id = 3021
X, y, cat_columns, num_columns = load_data(task_id)
X, label_encoders = preprocess_data(X, cat_columns)

# Encode the target variable
target_encoder = LabelEncoder()
y_encoded = target_encoder.fit_transform(y)  # Encode the target labels to integers

params = get_random_parameters(seed=42)
params['cat_idx'] = [X.columns.get_loc(c) for c in cat_columns]
params['num_idx'] = [X.columns.get_loc(c) for c in num_columns if c in X.columns]
params['target'] = np.unique(y_encoded)  # Add the unique targets for the output layer size

# Split the data
X_train, X_val, y_train, y_val = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Train the model
model, accuracy = train_model(X_train, y_train, X_val, y_val, params)
print(f'Validation Accuracy: {accuracy:.4f}')


Validation Accuracy: 0.9497


# Task ID70: 3481

In [ ]:
# Load and preprocess the data
task_id = 3481
X, y, cat_columns, num_columns = load_data(task_id)
X, label_encoders = preprocess_data(X, cat_columns)

# Encode the target variable
target_encoder = LabelEncoder()
y_encoded = target_encoder.fit_transform(y)  # Encode the target labels to integers

params = get_random_parameters(seed=42)
params['cat_idx'] = [X.columns.get_loc(c) for c in cat_columns]
params['num_idx'] = [X.columns.get_loc(c) for c in num_columns if c in X.columns]
params['target'] = np.unique(y_encoded)  # Add the unique targets for the output layer size

# Split the data
X_train, X_val, y_train, y_val = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Train the model
model, accuracy = train_model(X_train, y_train, X_val, y_val, params)
print(f'Validation Accuracy: {accuracy:.4f}')


Validation Accuracy: 0.9449


# Task ID71: 3573

In [ ]:
# Load and preprocess the data
task_id = 3573
X, y, cat_columns, num_columns = load_data(task_id)
X, label_encoders = preprocess_data(X, cat_columns)

# Encode the target variable
target_encoder = LabelEncoder()
y_encoded = target_encoder.fit_transform(y)  # Encode the target labels to integers

params = get_random_parameters(seed=42)
params['cat_idx'] = [X.columns.get_loc(c) for c in cat_columns]
params['num_idx'] = [X.columns.get_loc(c) for c in num_columns if c in X.columns]
params['target'] = np.unique(y_encoded)  # Add the unique targets for the output layer size

# Split the data
X_train, X_val, y_train, y_val = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Train the model
model, accuracy = train_model(X_train, y_train, X_val, y_val, params)
print(f'Validation Accuracy: {accuracy:.4f}')


Validation Accuracy: 0.9059


# Task ID72: 146824

In [ ]:
# Load and preprocess the data
task_id = 146824
X, y, cat_columns, num_columns = load_data(task_id)
X, label_encoders = preprocess_data(X, cat_columns)

# Encode the target variable
target_encoder = LabelEncoder()
y_encoded = target_encoder.fit_transform(y)  # Encode the target labels to integers

params = get_random_parameters(seed=42)
params['cat_idx'] = [X.columns.get_loc(c) for c in cat_columns]
params['num_idx'] = [X.columns.get_loc(c) for c in num_columns if c in X.columns]
params['target'] = np.unique(y_encoded)  # Add the unique targets for the output layer size

# Split the data
X_train, X_val, y_train, y_val = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Train the model
model, accuracy = train_model(X_train, y_train, X_val, y_val, params)
print(f'Validation Accuracy: {accuracy:.4f}')


Validation Accuracy: 0.9650


# Task ID73: 146820

In [ ]:
# Load and preprocess the data
task_id = 146820
X, y, cat_columns, num_columns = load_data(task_id)
X, label_encoders = preprocess_data(X, cat_columns)

# Encode the target variable
target_encoder = LabelEncoder()
y_encoded = target_encoder.fit_transform(y)  # Encode the target labels to integers

params = get_random_parameters(seed=42)
params['cat_idx'] = [X.columns.get_loc(c) for c in cat_columns]
params['num_idx'] = [X.columns.get_loc(c) for c in num_columns if c in X.columns]
params['target'] = np.unique(y_encoded)  # Add the unique targets for the output layer size

# Split the data
X_train, X_val, y_train, y_val = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Train the model
model, accuracy = train_model(X_train, y_train, X_val, y_val, params)
print(f'Validation Accuracy: {accuracy:.4f}')


Validation Accuracy: 0.9421


# Task ID74: 146822

In [ ]:
# Load and preprocess the data
task_id = 146822
X, y, cat_columns, num_columns = load_data(task_id)
X, label_encoders = preprocess_data(X, cat_columns)

# Encode the target variable
target_encoder = LabelEncoder()
y_encoded = target_encoder.fit_transform(y)  # Encode the target labels to integers

params = get_random_parameters(seed=42)
params['cat_idx'] = [X.columns.get_loc(c) for c in cat_columns]
params['num_idx'] = [X.columns.get_loc(c) for c in num_columns if c in X.columns]
params['target'] = np.unique(y_encoded)  # Add the unique targets for the output layer size

# Split the data
X_train, X_val, y_train, y_val = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Train the model
model, accuracy = train_model(X_train, y_train, X_val, y_val, params)
print(f'Validation Accuracy: {accuracy:.4f}')


Validation Accuracy: 0.8333


# Task ID75: 146195

In [ ]:
# Load and preprocess the data
task_id = 146195
X, y, cat_columns, num_columns = load_data(task_id)
X, label_encoders = preprocess_data(X, cat_columns)

# Encode the target variable
target_encoder = LabelEncoder()
y_encoded = target_encoder.fit_transform(y)  # Encode the target labels to integers

params = get_random_parameters(seed=42)
params['cat_idx'] = [X.columns.get_loc(c) for c in cat_columns]
params['num_idx'] = [X.columns.get_loc(c) for c in num_columns if c in X.columns]
params['target'] = np.unique(y_encoded)  # Add the unique targets for the output layer size

# Split the data
X_train, X_val, y_train, y_val = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Train the model
model, accuracy = train_model(X_train, y_train, X_val, y_val, params)
print(f'Validation Accuracy: {accuracy:.4f}')


Validation Accuracy: 0.6585


# Task ID76: 146800

In [ ]:
# Load and preprocess the data
task_id = 146800
X, y, cat_columns, num_columns = load_data(task_id)
X, label_encoders = preprocess_data(X, cat_columns)

# Encode the target variable
target_encoder = LabelEncoder()
y_encoded = target_encoder.fit_transform(y)  # Encode the target labels to integers

params = get_random_parameters(seed=42)
params['cat_idx'] = [X.columns.get_loc(c) for c in cat_columns]
params['num_idx'] = [X.columns.get_loc(c) for c in num_columns if c in X.columns]
params['target'] = np.unique(y_encoded)  # Add the unique targets for the output layer size

# Split the data
X_train, X_val, y_train, y_val = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Train the model
model, accuracy = train_model(X_train, y_train, X_val, y_val, params)
print(f'Validation Accuracy: {accuracy:.4f}')


Validation Accuracy: 0.1481


# Task ID77: 146817

In [ ]:
# Load and preprocess the data
task_id = 146817
X, y, cat_columns, num_columns = load_data(task_id)
X, label_encoders = preprocess_data(X, cat_columns)

# Encode the target variable
target_encoder = LabelEncoder()
y_encoded = target_encoder.fit_transform(y)  # Encode the target labels to integers

params = get_random_parameters(seed=42)
params['cat_idx'] = [X.columns.get_loc(c) for c in cat_columns]
params['num_idx'] = [X.columns.get_loc(c) for c in num_columns if c in X.columns]
params['target'] = np.unique(y_encoded)  # Add the unique targets for the output layer size

# Split the data
X_train, X_val, y_train, y_val = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Train the model
model, accuracy = train_model(X_train, y_train, X_val, y_val, params)
print(f'Validation Accuracy: {accuracy:.4f}')


Validation Accuracy: 0.4473


# Task ID78: 146819

In [ ]:
# Load and preprocess the data
task_id = 146819
X, y, cat_columns, num_columns = load_data(task_id)
X, label_encoders = preprocess_data(X, cat_columns)

# Encode the target variable
target_encoder = LabelEncoder()
y_encoded = target_encoder.fit_transform(y)  # Encode the target labels to integers

params = get_random_parameters(seed=42)
params['cat_idx'] = [X.columns.get_loc(c) for c in cat_columns]
params['num_idx'] = [X.columns.get_loc(c) for c in num_columns if c in X.columns]
params['target'] = np.unique(y_encoded)  # Add the unique targets for the output layer size

# Split the data
X_train, X_val, y_train, y_val = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Train the model
model, accuracy = train_model(X_train, y_train, X_val, y_val, params)
print(f'Validation Accuracy: {accuracy:.4f}')


Validation Accuracy: 0.9074


# Task ID79: 146821

In [ ]:
# Load and preprocess the data
task_id = 146821
X, y, cat_columns, num_columns = load_data(task_id)
X, label_encoders = preprocess_data(X, cat_columns)

# Encode the target variable
target_encoder = LabelEncoder()
y_encoded = target_encoder.fit_transform(y)  # Encode the target labels to integers

params = get_random_parameters(seed=42)
params['cat_idx'] = [X.columns.get_loc(c) for c in cat_columns]
params['num_idx'] = [X.columns.get_loc(c) for c in num_columns if c in X.columns]
params['target'] = np.unique(y_encoded)  # Add the unique targets for the output layer size

# Split the data
X_train, X_val, y_train, y_val = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Train the model
model, accuracy = train_model(X_train, y_train, X_val, y_val, params)
print(f'Validation Accuracy: {accuracy:.4f}')


Validation Accuracy: 0.6676


# Task ID80: 14954

In [ ]:
# Load and preprocess the data
task_id = 14954
X, y, cat_columns, num_columns = load_data(task_id)
X, label_encoders = preprocess_data(X, cat_columns)

# Encode the target variable
target_encoder = LabelEncoder()
y_encoded = target_encoder.fit_transform(y)  # Encode the target labels to integers

params = get_random_parameters(seed=42)
params['cat_idx'] = [X.columns.get_loc(c) for c in cat_columns]
params['num_idx'] = [X.columns.get_loc(c) for c in num_columns if c in X.columns]
params['target'] = np.unique(y_encoded)  # Add the unique targets for the output layer size

# Split the data
X_train, X_val, y_train, y_val = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Train the model
model, accuracy = train_model(X_train, y_train, X_val, y_val, params)
print(f'Validation Accuracy: {accuracy:.4f}')


Validation Accuracy: 0.4444


# Task ID81: 167141

In [ ]:
# Load and preprocess the data
task_id = 167141
X, y, cat_columns, num_columns = load_data(task_id)
X, label_encoders = preprocess_data(X, cat_columns)

# Encode the target variable
target_encoder = LabelEncoder()
y_encoded = target_encoder.fit_transform(y)  # Encode the target labels to integers

params = get_random_parameters(seed=42)
params['cat_idx'] = [X.columns.get_loc(c) for c in cat_columns]
params['num_idx'] = [X.columns.get_loc(c) for c in num_columns if c in X.columns]
params['target'] = np.unique(y_encoded)  # Add the unique targets for the output layer size

# Split the data
X_train, X_val, y_train, y_val = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Train the model
model, accuracy = train_model(X_train, y_train, X_val, y_val, params)
print(f'Validation Accuracy: {accuracy:.4f}')


Validation Accuracy: 0.8620


# Task ID82: 167140

In [ ]:
# Load and preprocess the data
task_id = 167140
X, y, cat_columns, num_columns = load_data(task_id)
X, label_encoders = preprocess_data(X, cat_columns)

# Encode the target variable
target_encoder = LabelEncoder()
y_encoded = target_encoder.fit_transform(y)  # Encode the target labels to integers

params = get_random_parameters(seed=42)
params['cat_idx'] = [X.columns.get_loc(c) for c in cat_columns]
params['num_idx'] = [X.columns.get_loc(c) for c in num_columns if c in X.columns]
params['target'] = np.unique(y_encoded)  # Add the unique targets for the output layer size

# Split the data
X_train, X_val, y_train, y_val = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Train the model
model, accuracy = train_model(X_train, y_train, X_val, y_val, params)
print(f'Validation Accuracy: {accuracy:.4f}')


Validation Accuracy: 0.9342


# Task ID83: 167125

In [ ]:
# Load and preprocess the data
task_id = 167125
X, y, cat_columns, num_columns = load_data(task_id)
X, label_encoders = preprocess_data(X, cat_columns)

# Encode the target variable
target_encoder = LabelEncoder()
y_encoded = target_encoder.fit_transform(y)  # Encode the target labels to integers

params = get_random_parameters(seed=42)
params['cat_idx'] = [X.columns.get_loc(c) for c in cat_columns]
params['num_idx'] = [X.columns.get_loc(c) for c in num_columns if c in X.columns]
params['target'] = np.unique(y_encoded)  # Add the unique targets for the output layer size

# Split the data
X_train, X_val, y_train, y_val = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Train the model
model, accuracy = train_model(X_train, y_train, X_val, y_val, params)
print(f'Validation Accuracy: {accuracy:.4f}')


Validation Accuracy: 0.9573


# Task ID84: 167124

# Task ID85: 167121

In [ ]:
pip freeze > requirements.txt